In [1]:
import  pandas as pd 
import warnings
import numpy as np
warnings.filterwarnings("ignore")
import datetime
today=datetime.date.today()
str_today=today.strftime('%Y%m%d')

## SKU 对应表

In [2]:
sap=pd.read_excel(r"\\192.168.184.120\基础数据\SAP-SKU\SAP_SKU.xlsx")

sap.columns

Index(['SAP编码', 'SAP名称', 'SKU', 'Asin', 'MODEL', 'Class1', 'Class2', 'Market',
       'Platform', '备注', 'Unnamed: 10', 'Unnamed: 11'],
      dtype='object')

In [3]:
sap=sap[sap['Platform']=="Walmart"]
# 获取sku表里platform字段为walmart的所有数据

In [4]:
sap['sap']=sap['SAP编码'].map(lambda x:x.strip())
sap['sku']=sap['SKU'].map(lambda x:x.strip())
sap=sap[['sap','sku']].drop_duplicates()
sap.head()

,sap,sku
296,8.01.0276,WM-S45Max-01
297,8.03.0012,VM-H6-01
298,8.01.0162,WM-S65Pure-01
299,8.01.0239,VM-S6MaxV-01
300,8.01.0030,WM-E351-01


## payment 数据

### 数据合并

In [5]:
pay1=pd.read_csv(r"E:/DATA/Walmart/Payment/Legacy/Roborock_Technology_Co__10001070788_MP_02282023_reconciliationreport.csv",header=0) 
pay2=pd.read_csv(r"E:/DATA/Walmart/Payment/Legacy/Roborock_Technology_Co__10001070788_MP_02142023_reconciliationreport.csv",header=0) 
pay3=pd.read_csv(r"E:/DATA/Walmart/Payment/Legacy/Roborock_Technology_Co__10001070788_MP_02072023_reconciliationreport.csv",header=0) 

pay = pd.concat([pay1,pay2,pay3])

In [6]:
pay['Walmart.com Order #']=pay['Walmart.com Order #'].astype(str)
pay['Walmart.com PO #']=pay['Walmart.com PO #'].astype(str)
pay["Transaction Date Time"]=pd.to_datetime(pay["Transaction Date Time"].map(lambda x:x[-4:]+'/'+x[:-5]))
pay = pay[pay['Fulfillment Type']=="Walmart-fulfilled(WFS)"]

In [7]:
print(pay['Transaction Date Time'].min(),pay['Transaction Date Time'].max())

2023-01-28 00:00:00 2023-02-24 00:00:00


In [8]:
pay['Transaction Type'].unique()

array(['SALE', 'ADJMNT', 'REFUNDED'], dtype=object)

In [9]:
pay3 = pd.merge(pay,sap,how='left',left_on='Partner Item ID',right_on='sku')

In [10]:
df_payment = pay3[['Walmart.com Order #', 'Walmart.com PO #','Partner Item ID','Adjustment Description',
     'Transaction Type','Transaction Date Time','Fulfillment Type','Shipped Qty','Payable to Partner from Sale','Gross Sales Revenue','Refunded Retail Sales','sap']]

### 数据源输出

In [11]:
# 可以用这份原始数据，做数据透视表和结果进行比对
df_payment.to_excel(r"F:\Walmart\Walmart_payment\Walmart\过程数据\payment%s.xlsx"%str_today,index=False)

### 数据清洗

#### 销售订单数据

In [66]:
df_sale = df_payment[df_payment['Transaction Type'] == 'SALE']
df_payment_group_by_sap = df_sale.groupby(['sap'])['Shipped Qty','Payable to Partner from Sale','Gross Sales Revenue','Refunded Retail Sales'].sum()
df_payment_group_by_sap
df_payment_group_by_sap['订单情况'] = ""
df_payment_group_by_sap['output'] = df_payment_group_by_sap['Gross Sales Revenue'] 
df_payment_group_by_sap

,Shipped Qty,Payable to Partner from Sale,Gross Sales Revenue,Refunded Retail Sales,订单情况,output
sap,,,,,,
8.01.0162,1,305.99,359.99,0.0,,359.99
8.01.0298,7,1213.93,1399.93,0.0,,1399.93
8.01.0306,26,9838.14,11179.74,0.0,,11179.74
8.01.0307,28,10582.02,12039.72,0.0,,12039.72
8.01.0320,13,3431.87,3899.87,0.0,,3899.87
8.01.0324,1,158.39,179.99,0.0,,179.99
8.01.0343,14,3212.86,3779.86,0.0,,3779.86
8.01.0379,19,11487.21,13389.81,0.0,,13389.81
8.01.0381,33,20270.87,23299.67,0.0,,23299.67


#### 退款订单数据

In [67]:
df_refund = df_payment[df_payment['Transaction Type'] == 'REFUNDED']
df_refund['Refunded Retail Sales'] = df_refund['Refunded Retail Sales'].abs()
#因为原始数据中，如果是refund的订单，数量默认是0
max_values = df_refund.groupby('sap')['Refunded Retail Sales'].max()   
# 需要区分是部分退货还是全款退货
refund_all = df_refund[(df_refund['Refunded Retail Sales'] / df_refund['sap'].map(max_values)) > 0.4 ]
refund_all['Shipped Qty'] = 1
refund_part = df_refund[(df_refund['Refunded Retail Sales'] / df_refund['sap'].map(max_values)) <= 0.4 ]
refund_part['Shipped Qty'] = 0

In [68]:
# 正常退款部分
refund_all_gb = refund_all.groupby(['sap'])['Shipped Qty','Payable to Partner from Sale','Gross Sales Revenue','Refunded Retail Sales'].sum()
refund_all_gb['退款类型'] = "全额退款"
refund_all_gb['output'] = refund_all_gb['Refunded Retail Sales']
refund_all_gb.head()

,Shipped Qty,Payable to Partner from Sale,Gross Sales Revenue,Refunded Retail Sales,退款类型,output
sap,,,,,,
8.01.0162,1,-316.79,0.0,359.99,全额退款,359.99
8.01.0298,5,-873.95,0.0,999.95,全额退款,999.95
8.01.0306,5,-1871.35,0.0,2107.95,全额退款,2107.95
8.01.0307,8,-2939.12,0.0,3339.92,全额退款,3339.92
8.01.0324,2,-334.38,0.0,379.98,全额退款,379.98


In [69]:
#部分退款数据
refund_part_gb = refund_part.groupby(['sap'])['Shipped Qty','Payable to Partner from Sale','Gross Sales Revenue','Refunded Retail Sales'].sum()
refund_part_gb['退款类型'] = "部分退款"
refund_part_gb['output'] = refund_part_gb['Refunded Retail Sales']
refund_part_gb

,Shipped Qty,Payable to Partner from Sale,Gross Sales Revenue,Refunded Retail Sales,退款类型,output
sap,,,,,,
8.01.0306,0,-16.45,0.0,18.69,部分退款,18.69
8.01.0379,0,-19.36,0.0,22.78,部分退款,22.78


#### 正值调整

In [70]:
df_adjment_sale = df_payment.loc[((df_payment['Transaction Type'] == 'ADJMNT') & (df_payment['Payable to Partner from Sale'] > 0))]

df_adjment_sale_gb = df_adjment_sale.groupby(['sap'])['Shipped Qty','Payable to Partner from Sale','Gross Sales Revenue','Refunded Retail Sales'].sum()
df_adjment_sale_gb['output'] = df_adjment_sale_gb['Payable to Partner from Sale']
df_adjment_sale_gb['订单情况'] = "退货服务费"
df_adjment_sale_gb

,Shipped Qty,Payable to Partner from Sale,Gross Sales Revenue,Refunded Retail Sales,output,订单情况
sap,,,,,,


#### 负值调整

In [71]:
df_adjment_refund = df_payment.loc[((df_payment['Transaction Type'] == 'ADJMNT') & (df_payment['Payable to Partner from Sale'] < 0))]

df_adjment_refund_gb = df_adjment_refund.groupby(['sap'])['Shipped Qty','Payable to Partner from Sale','Gross Sales Revenue','Refunded Retail Sales'].sum()
df_adjment_refund_gb['Shipped Qty'] = 0
df_adjment_refund_gb['output'] = df_adjment_refund_gb['Payable to Partner from Sale'].abs()
df_adjment_refund_gb['订单情况'] = "退货服务费"
df_adjment_refund_gb['退款类型'] = "调整服务费"

### 销售订单合并

In [72]:
order = pd.concat([df_payment_group_by_sap,df_adjment_sale_gb])

In [73]:
order.head()

,Shipped Qty,Payable to Partner from Sale,Gross Sales Revenue,Refunded Retail Sales,订单情况,output
sap,,,,,,
8.01.0162,1,305.99,359.99,0.0,,359.99
8.01.0298,7,1213.93,1399.93,0.0,,1399.93
8.01.0306,26,9838.14,11179.74,0.0,,11179.74
8.01.0307,28,10582.02,12039.72,0.0,,12039.72
8.01.0320,13,3431.87,3899.87,0.0,,3899.87


In [74]:
order = order[['Shipped Qty',"output","订单情况"]]

### 退货订单合并

In [75]:
refund = pd.concat([refund_all_gb,refund_part_gb,df_adjment_refund_gb])

In [76]:
refund.head()

,Shipped Qty,Payable to Partner from Sale,Gross Sales Revenue,Refunded Retail Sales,退款类型,output,订单情况
sap,,,,,,,
8.01.0162,1,-316.79,0.0,359.99,全额退款,359.99,NaN
8.01.0298,5,-873.95,0.0,999.95,全额退款,999.95,NaN
8.01.0306,5,-1871.35,0.0,2107.95,全额退款,2107.95,NaN
8.01.0307,8,-2939.12,0.0,3339.92,全额退款,3339.92,NaN
8.01.0324,2,-334.38,0.0,379.98,全额退款,379.98,NaN


In [77]:
refund = refund[['Shipped Qty',"output","订单情况","退款类型"]]

## 数据匹配模板

### 销售账单输出

In [78]:
df5=order
df5['promotional rebates']=0

In [79]:
#获取上个月的月末
today = datetime.date.today()
first_day_of_month = today.replace(day=1)
last_day_of_prev_month = first_day_of_month - datetime.timedelta(days=1)
last_day_of_prev_month_str = last_day_of_prev_month.strftime("%Y%m%d")
aa ="US"+last_day_of_prev_month_str

In [80]:
df5["订单类型"]='ZOR9'

df5["订单原因"]=""
df5["订单原因"][df5['Shipped Qty']==0]='退货服务费'

df5["库存地点"]='NA01'

df5["销售组织"]="4200"

df5["分销渠道"]="30"

df5['产品组']="00"

df5['销售部门']="3000"

df5['销售组']="Z10"

df5['售达方']="100177"

df5['送达方']="100177"

df5['采购订单编号'] = aa

df5['订单日期']=last_day_of_prev_month_str     

df5['工厂']="4200"

df5["国贸条款版本"]=""
df5["国际贸易条款"]=""
df5["抬头注释"]=""
df5['币别']="USD"
df5['（项目）类别']=""
df5 = df5.reset_index()

In [81]:
df5 = df5.rename(columns={'sap':'物料','Shipped Qty':"数量",'output':"价格（总价）",'promotional rebates':'折扣（总价）'})

In [82]:
df5.columns

Index(['物料', '数量', '价格（总价）', '订单情况', '折扣（总价）', '订单类型', '订单原因', '库存地点', '销售组织',
       '分销渠道', '产品组', '销售部门', '销售组', '售达方', '送达方', '采购订单编号', '订单日期', '工厂',
       '国贸条款版本', '国际贸易条款', '抬头注释', '币别', '（项目）类别'],
      dtype='object')

In [83]:
df6=df5[['订单类型','销售组织','分销渠道','产品组',
         '销售部门', '销售组', '售达方', '送达方', 
         '采购订单编号', '订单日期','订单原因',
        '国贸条款版本','国际贸易条款',
         '抬头注释','物料','数量','工厂',
        '库存地点',"价格（总价）",'折扣（总价）','币别','（项目）类别',"订单情况"]]

In [84]:
df6.to_excel(r"F:\Walmart\Walmart_payment\Walmart\过程数据\Walmart销售订单%s.xlsx"%str_today,index=False,sheet_name='销售订单')

### 退货账单输出

In [85]:
da5=refund

In [86]:
da5['promotional rebates']=0

In [87]:
da5 = da5.reset_index()

In [88]:
da5.rename(columns={'sap':'物料','Shipped Qty':"数量",'output':"价格（总价）",'promotional rebates':'折扣（总价）'},inplace=True)

In [89]:
#获取上个月的月初
today = datetime.date.today()  # 获取今天的日期
first_day_of_this_month = datetime.date(today.year, today.month, 1)  # 获取本月月初的日期
first_day_of_last_month = first_day_of_this_month - datetime.timedelta(days=1)  # 上个月月末的日期
first_day_of_last_month = datetime.date(first_day_of_last_month.year, first_day_of_last_month.month, 1)  # 上个月月初的日期
result = first_day_of_last_month.strftime('%Y%m%d')
bb ="US"+result

In [90]:
da5["订单类型"]='ZRE3'

da5["订单原因"]=""

da5["库存地点"]='NA01'

da5["销售组织"]="4200"

da5["分销渠道"]="30"

da5['产品组']="00"

da5['销售部门']="7000"

da5['销售组']="Z10"

da5['售达方']="100177"

da5['送达方']="100177"

da5['采购订单编号']=bb #站点加日期

da5['订单日期']= result 

da5['工厂']="4200"

da5["国贸条款版本"]=""
da5["国际贸易条款"]=""
da5["抬头注释"]=""

da5['币别']="USD"
da5['（项目）类别']=""

In [91]:
da6=da5[['订单类型','销售组织','分销渠道','产品组',
         '销售部门', '销售组', '售达方', '送达方', 
         '采购订单编号', '订单日期','订单原因',
        '国贸条款版本','国际贸易条款',
         '抬头注释','物料','数量','工厂',
        '库存地点',"价格（总价）",'折扣（总价）','币别','（项目）类别','退款类型',"订单情况"]]

In [92]:
da6.to_excel(r"F:\Walmart\Walmart_payment\Walmart\过程数据\Walmart退货订单%s.xlsx"%str_today,index=False,sheet_name='退货订单')